In [1]:

import numpy as np
import matplotlib.pylab as plt
import time
from scipy import optimize
from scipy import integrate
# from numba import jit
import pandas as pd

In [2]:
Dim = 3
x = np.random.normal(0,1,(1000,Dim))
x[:,1] += 2.
x[:,2] = 2*x[:,2]

print(x)

[[-0.43741237  2.45030949 -2.54983045]
 [-0.25222277  0.37845323  0.4760773 ]
 [ 0.11652006  3.21789698  1.14139801]
 ...
 [-0.04426879  3.21549524 -0.61888039]
 [ 0.82433164  2.84781587  0.5871924 ]
 [-0.46513324  3.46793563  0.5216905 ]]


In [3]:
Library = np.ones((x.shape[0],1))
print(Library.shape)


for i in range(Dim): # linear
    Library = np.concatenate((Library,x[:,i].reshape(-1,1)),
                             axis = 1)
    
for i in range(Dim): # quadratic
    for j in range(i,Dim):
        Library = np.concatenate((Library,(x[:,i]*x[:,j]).reshape(-1,1)),
                             axis = 1)    

(1000, 1)


In [4]:
def D1(x):
    return(x[:,0] + 2*x[:,0]**2. + 0.01 * x[:,1]- 0.5*x[:,2]**2.)

D1(x[0,:].reshape(1,-1))

array([-3.28106779])

In [5]:
D_on_x = D1(x)
print(D_on_x.shape, type(D_on_x))
print(D_on_x)

(1000,) <class 'numpy.ndarray'>
[-3.28106779e+00 -2.34530386e-01 -4.75541831e-01 -4.20853889e-01
  2.60216003e-02 -1.54652346e+00 -8.42221551e-01 -7.58512320e+00
 -3.68204494e-01  2.11159358e-01 -1.49497542e+00 -5.02154732e-01
 -7.38938019e-01 -5.04167375e+00  2.63689407e+00 -8.65367897e-01
 -1.72514035e+00 -3.91693050e+00  1.46235701e+00  6.35173947e+00
 -4.40451002e+00 -2.57671682e+00  7.07956313e-01 -4.90527432e-01
  1.12897574e-02  6.32142688e+00  1.22189053e+00  4.34124727e-01
 -2.96736307e-01 -7.19018887e-01 -1.16967184e-01 -1.42926090e-01
 -1.17995113e-01 -7.77206539e-01 -8.69876932e+00  8.88057731e-01
  9.93446752e-02  4.47977788e-01 -1.55959733e+00 -8.86918574e-01
  9.96614866e-01  4.10536152e-01  1.95550123e+00  4.26327279e+00
 -4.96741105e-01  3.76650701e+00  5.10942867e-01  7.26470061e-01
 -1.00908831e+01  6.50793576e-01  3.79713591e-01  2.39556489e+00
  6.59696506e+00  2.85947164e+00  6.91836834e+00  2.89759544e-01
  5.48749689e+00 -7.78955247e-01  4.98897974e-01 -6.332411

### Ranges


In [6]:
x[0,:]

array([-0.43741237,  2.45030949, -2.54983045])

In [7]:
Ranges = np.empty((Dim,2)) # minimum and Maximum
for i in range(Dim):
    Ranges[i,:] = [min(x[:,i]), max(x[:,i])]
print(Ranges)

[[-2.82979604  3.04992814]
 [-1.52535385  4.83684162]
 [-8.44448199  6.21481021]]


### Simple Integral Approach

In [8]:
def Marginalise_over_One(x,i,f, Range_i, nx=100):
    # input: x0, x1, ....; i is the xi for which f is integrated over
    #        nx: number of increments for integration
    # note that the value for x_i is irrelevant for the algorithm
    # output: marginalised value for function f with respect to xi    
    Support = np.linspace(Range_i[0], Range_i[1], nx)
    dxi = Support[1] - Support[0]
    
    dimx = len(x)
    IntegrationArea = np.empty((nx,dimx))
    for j in range(dimx):
        if j != i: # for all xj input: always constant values
            IntegrationArea[:,j] = np.ones(nx)*x[j]
        elif j == i:
            IntegrationArea[:,j] = Support
            
    f_val = f(IntegrationArea)
    return(f_val.sum()*dxi /(Ranges[i,1]- Ranges[i,0]) )



In [9]:
print(Marginalise_over_One(np.array([1.,1.,1.]),
                    i = 1, f = D1, Range_i = Ranges[1,:], nx=100))
print(Marginalise_over_One(np.array([1.,1.,1.]),
                    i = 1, f = D1, Range_i = Ranges[1,:] ,nx = 1000))
print(Marginalise_over_One(np.array([1.,1.,1.]),
                    i = 1, f = D1,Range_i = Ranges[1,:], nx = 100000))

2.5419772109618464
2.5190765153675807
2.5165826046750257


In [10]:
def Integrand(x,i,f,Range_i, nx=100):
    return( (f(x.reshape(1,-1)) - Marginalise_over_One( x,i, f, Range_i, nx=100  ))**2. )

In [11]:
print(Integrand(np.array([1.,1.,1.]),
                    i = 0, f = D1, Range_i = Ranges[0,:], nx=100))

[9.41470485]


In [12]:

def One(x0,x1,x2):
    return(1.)

def integrand(x0,x1,x2,i, f, Range_i, nx):
    return(Integrand(np.array([x0,x1,x2]), i, f , Range_i , nx))

In [13]:
Volume, verror = integrate.nquad(One,[[-3,3], [-1,5], [-7,6]])
print(Volume, verror)
for i in range(Dim):
    
    result, error = integrate.nquad(integrand, [[-3,3], [-1,5], [-7,6]],
                               args = (i, D1, Ranges[i,:], 100)) 
    print(i, result/Volume, error)

468.0 5.195843755245733e-12
0 31.804407820535996 1.652507337636205e-10
1 0.008177945069799427 4.2491324821659986e-14
2 50.09506402240092 2.60286125569427e-10


In [14]:
Integrands = list([Integrand, integrand])
print(Integrands)

[<function Integrand at 0x7f09fb89b1e0>, <function integrand at 0x7f09fb89b9d8>]


## Now with the 6D-Coupled-Lorenz

In [15]:

def integrand6(x0,x1,x2,x3,x4,x5,x6,i, f, Range_i, nx):
    return(Integrand(np.array([x0,x1,x2,x3,x4,x5,x6]), i, f , Range_i , nx))

In [16]:
BIC= pd.read_csv("Lorenz_Score_BIC.csv", index_col = 0)
print(BIC)
print(BIC["BIC"]==min(BIC["BIC"]))
logic = BIC["BIC"]==min(BIC["BIC"])
print(np.arange(3)[logic])

           BIC  Threshold
0  1876.523805       0.05
1  1747.523895       0.55
2  7867.645786       1.05
0    False
1     True
2    False
Name: BIC, dtype: bool
[1]


In [17]:

filename = 'CoupledLorenz_dt0.01' 
x = np.load(filename+'.npy')
print(x.shape)

(5000, 6)


In [18]:
BestAlphas = np.loadtxt('CoupledLorenz_Alpha_Done.txt')[BIC["BIC"]==min(BIC["BIC"]),:]

In [19]:
RangeL = np.empty((6,2))
Dim = x.shape[1]    # NOW NEW DIMENSION!
for i in range(Dim):
    RangeL[i,:] = [min(x[:,i]), max(x[:,i])] 
print(RangeL)

[[-19.32628135  21.26101002]
 [-24.5107052   27.94186795]
 [  1.          52.2962196 ]
 [-22.45140621  20.64704473]
 [-28.70851393  24.51948398]
 [  3.74703334  53.52157254]]


In [20]:
# let's only regard up to second order    
def poly(x,sigma):
    x_vec=np.array([1,x[0],x[1],x[2],x[3],x[4],x[5],  #7
                   x[0]**2., x[1]**2., x[2]**2., x[3]**2., x[4]**2., x[5]**2., # 6 terms
                    x[0]*x[1], x[0]*x[2], x[0]*x[3], x[0]*x[4], x[0]*x[5], # 5 terms
                    x[1]*x[2], x[1]*x[3], x[1]*x[4], x[1]*x[5], # 4 terms
                    x[2]*x[3], x[2]*x[4], x[2]*x[5], # 3 terms
                    x[3]*x[4], x[3]*x[5], x[4]*x[5]]) # 3 terms
    
    return np.dot(sigma,x_vec) # Total: 28 terms

#@jit
def D1(sigma,x):
    sigma = sigma[1:] # without noise parameters
    sigma=sigma.reshape((Dim,-1))
    function=np.zeros((len(x),Dim))
    for i in range(Dim):
        function[:,i]=poly(x.T,sigma[i])
    return function

In [21]:
poly(x.T,BestAlphas.flatten()[1:29])

/home/users/t_wand01/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


array([  0.91615775,  -2.33633244,   4.81708547, ...,  -6.31656045,
        -8.84006699, -11.14266855])

In [22]:
print(BestAlphas.shape)
print(BestAlphas.flatten().shape)


(1, 169)
(169,)


In [23]:
def D1_0(x0,x1,x2,x3,x4,x5):
    # returns the j'th component of D1
    x_arr = np.array([x0,x1,x2,x3,x4,x5])
    return(D1(Sigma,x_arr)[0,0])

for j in range(6):
    print(D1_j(x[0,0], x[0,1],x[0,2], x[0,3],x[0,4],x[0,5],
               BestAlphas.flatten(),j))

0.9161577529163942
26.622084909093488
0.1684962347598935
15.586333694423445
161.0531733434817
76.04350914923002


In [24]:
def One6(x0,x1,x2,x3,x4,x5):
    return(1)

In [25]:
Volume, verror = integrate.nquad(One6,RangeL)
print(Volume, verror)


12469563969.440876 0.00013843997025909523


In [26]:
print(RangeL)
V = 1
for i in range(6):
    V = V*(RangeL[i,1]-RangeL[i,0])
print(V)

[[-19.32628135  21.26101002]
 [-24.5107052   27.94186795]
 [  1.          52.2962196 ]
 [-22.45140621  20.64704473]
 [-28.70851393  24.51948398]
 [  3.74703334  53.52157254]]
12469563969.440868


In [27]:
Marginalise_over_One

<function __main__.Marginalise_over_One(x, i, f, Range_i, nx=100)>

In [ ]:
print(Marginalise_over_One(np.array([1.,1.,1.,1.,1.,1.]),
                    i = 1, f = D1, Range_i = Ranges[1,:], nx=100))

In [28]:
for i in range(Dim):
    
    print(i, Marginalise_over_One(x, i=0, D1_j()))

SyntaxError: positional argument follows keyword argument (978212723.py, line 3)

In [ ]:
def One2d(x0,x1):
    return([1,1])
print(One2d(1,1))
print(integrate.nquad(One2d,[[-2,1.5], [0,3]]))